# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [3]:
! pip install -q schedule pytest # установка библиотек, если ещё не


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [17]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
import datetime
import requests
import schedule
import json
from typing import Tuple
from bs4 import BeautifulSoup

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [5]:
def get_response(url: str) -> Tuple[requests.Response | None, int | None]:
    """
    Отправляет и получает ответ на GET запрос. Проверяет статусы ответов.

    Args:
        url: адрес страницы

    Returns:
        requests.Response в случае кода ответа 2хх
            None во всех остальных случаях
        Код HTTP ответа или None при requests.exceptions.RequestException

    Raises:
        requests.exceptions.RequestException: Ошибка HTTP запроса
            подобная ConnectionError, Timeout, HTTPError, SSLError итд.
    """
    status_code = None
    try:
        response = requests.get(url)
        if 200 <= response.status_code < 300:
            return response, response.status_code
        else:
            status_code = response.status_code
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")

    return None, status_code


def get_book_data(book_url: str) -> dict:
    """
    Собирает со страницы название книги, цену, рейтинг, количество в наличии,
    описание, характеристики из таблицы Product Information.

    Args:
        book_url: ссылка на страницу с информацией о книге.

    Returns:
        словарь с характеристиками книги
            title
            price
            availability
            star_rating
            description
            upc
            product_type
            price_without_tax
            price_with_tax
            tax_rate
            reviews_number
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    book_dict = {}
    response, status_code = get_response(book_url)
    if response:
        soup = BeautifulSoup(response.content, 'html.parser', from_encoding='utf-8')
        product_page = soup.find('article', attrs={'class': 'product_page'})
        product_row = product_page.find('div', attrs={'class': 'row'})
        # title
        book_title = product_row.find('h1').text
        book_dict['title'] = book_title
        # price
        price = product_row.find('p', attrs={'class': 'price_color'}).text
        book_dict['price'] = price
        # availability
        availability = product_row.find('p', attrs={'class': 'instock availability'}).text.strip()
        book_dict['availability'] = availability
        # star-rating Four
        star_rating = product_row.find('p', attrs={'class': 'star-rating'})
        star_rating_class = star_rating.attrs['class']
        book_dict['star_rating'] = len(star_rating_class) == 2 and star_rating_class[1]
        # description
        product_description = product_page.find('div', id="product_description")
        book_dict['description'] = product_description and product_description.find_next_sibling('p').text
        # Product Information
        product_information = product_page.find('table', attrs={'class': "table table-striped"})
        table_ths = product_information.find_all('th')
        for th in table_ths:
            # upc
            if th.string == 'UPC':
                book_dict['upc'] = th.find_next_sibling('td').text
            # product_type
            elif th.string == 'Product Type':
                book_dict['product_type'] = th.find_next_sibling('td').text
            # price_without_tax
            elif th.string == 'Price (excl. tax)':
                book_dict['price_without_tax'] = th.find_next_sibling('td').text
            # price_with_tax
            elif th.string == 'Price (incl. tax)':
                book_dict['price_with_tax'] = th.find_next_sibling('td').text
            # tax_rate
            elif th.string == 'Tax':
                book_dict['tax_rate'] = th.find_next_sibling('td').text
            # reviews_number
            elif th.string == 'Number of reviews':
                book_dict['reviews_number'] = th.find_next_sibling('td').text
    else:
        print(f"Ошибка ответа от сервера. Код HTTP ответа: {status_code}")

    return book_dict
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [6]:
# Используйте для самопроверки
book_url = 'https://books.toscrape.com/catalogue/the-story-of-art_500/index.html'
book_dict = get_book_data(book_url)
print(book_dict['title'])


The Story of Art


## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [20]:
BASE_CATALOGUE_URL = "https://books.toscrape.com/catalogue/"
BASE_CATALOGUE_PAGE_URL = "https://books.toscrape.com/catalogue/page-{N}.html"


def get_books_links(url: str) -> list:
    """
    Собирает со страницы каталога ссылки на страницы с книгами.

    Args:
        url: ссылка на страницу каталога.

    Returns:
        Список ссылок на страницы с книгами
    """
    links = []
    response, status_code = get_response(url)
    if response:
        soup = BeautifulSoup(response.content, 'html.parser', from_encoding='utf-8')
        ordered_list = soup.find('ol', attrs={'class': 'row'})
        h3_tags = ordered_list.find_all('h3')
        for h3 in h3_tags:
            links.append(h3.find('a')['href'])

    return links


def get_catalogue_pages(url: str) -> dict:
    """
    Собирает страницы каталога в словарь.

    Args:
        url: ссылка шаблон страницы каталога вида
            https://books.toscrape.com/catalogue/page-{N}.html

    Returns:
        Словарь со ссылками на все страницы каталога
    """
    catalogue_pages = {}
    page_num = 1
    while links := get_books_links(url.format(N=page_num)):
        catalogue_pages[page_num] = links
        page_num += 1

    return catalogue_pages


def runtime(func):
    """
    Декоратор для замера времени выполнения парсинга.
    """
    def runtime_wrapper(*args, **kwargs):
        start_datetime = datetime.datetime.now()
        result = func(*args, **kwargs)
        end_datetime = datetime.datetime.now()
        time_difference = end_datetime - start_datetime
        total_seconds = time_difference.total_seconds()
        hours, remainder = divmod(total_seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        print(f"Время работы: {int(hours):02}:{int(minutes):02}:{seconds:05.2f}")
        return result

    return runtime_wrapper


@runtime
def scrape_books(is_save: bool = True) -> list[dict]:
    """
    Собирает информацию о книгах на страницах каталога и в зависимости от
    настройки выводит результаты в файл, и возвращает как результат работы

    Args:
        is_save: аргумент-флаг, который отвечает за сохранение результата
            в файл, если он будет равен `True`, то информация сохраняется
            построчно с конвертацией словаря в json
            в ту же папку в файл `books_data.txt`.

    Returns:
        Список ссылок на страницы с книгами
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    books = []

    for page, links in get_catalogue_pages(BASE_CATALOGUE_PAGE_URL).items():
        for link in links:
            book_dict = get_book_data(BASE_CATALOGUE_URL + link)
            books.append(book_dict)

    if is_save:
        with open('artifacts/books_data.txt', 'w', encoding='UTF-8') as file:
            for book in books:
                file.write(json.dumps(book, ensure_ascii=False) + '\n')

    return books
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [18]:
# Проверка работоспособности функции
res = scrape_books(is_save=True) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

Время работы: 00:09:55.84
<class 'list'> 1000


## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [28]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
def scrape_job():
    print(f"Парсинг стартовал в {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}")
    scrape_books()


schedule.every().day.at("19:00").do(scrape_job)

while True:
    schedule.run_pending()
    time.sleep(30)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

Парсинг стартовал в 2025-11-09 12:04:08
Время работы: 00:09:49.52


## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [32]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! pytest

]9;4;3;\============================= test session starts ==============================
platform linux -- Python 3.12.11, pytest-9.0.0, pluggy-1.6.0
rootdir: /home/george/projects/python/HW-03-VSC
plugins: anyio-4.11.0
collected 5 items                                                              

tests/test_scraper.py ]9;4;1;0\.]9;4;1;20\.]9;4;1;40\.]9;4;1;60\.]9;4;1;80\.                                              [100%]]9;4;0;\

======================== 5 passed in 604.17s (0:10:04) =========================


## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```